In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("Auto_Reply_v3.db")

# Load Q&A pairs (assuming table name is 'All_Reviews')
query1 = 'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews'
query2 = 'SELECT "translation" AS Review, "Reply" FROM "Auto Reply Hadith"'
query3 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews'
query4 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews_Al_Quran'

df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)
df3 = pd.read_sql_query(query3, conn)
df4 = pd.read_sql_query(query4, conn)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Close connection
conn.close()

# Check data
df.head()

# craete a csv file 
# df.to_csv("All_Review_Reply.csv", index=False)

,Review,Reply
0,"Alhamdulillah.. the best al quran application,...",None
1,the best quraan app you can find. it literally...,None
2,Very helpful,None
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...
4,May Allah reward you richly rewarded,None


In [2]:
import torch
from transformers import pipeline

/home/tahlilmahfuz/RAG_Implementation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check CUDA availability and set device
cuda_available = torch.cuda.is_available()
if cuda_available:
    device = 0  # GPU
    gpu_name = torch.cuda.get_device_name(device)
    print(f"CUDA is available. Using GPU: {gpu_name}")
else:
    device = -1  # CPU
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [4]:
# Initialize sentiment-analysis pipeline (uses SST-2 fine-tuned DistilBERT)
sentiment_analyzer = pipeline(
    'sentiment-analysis',
    model='distilbert-base-uncased-finetuned-sst-2-english',
    device=device
)

# Function to analyze sentiment (truncates to 512 chars to respect token limits)
def analyze_sentiment(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return None
    snippet = text[:512]
    result = sentiment_analyzer(snippet)
    return result[0]['label']

# Apply sentiment analysis to the 'Review' column
df['sentiment'] = df['Review'].apply(analyze_sentiment)

# Save the modified DataFrame to CSV
df.to_csv('modifiedDataset.csv', index=False)

print(f"Processed {len(df)} rows. Results saved to 2LableSentiment.csv.")


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 70460 rows. Results saved to 2LableSentiment.csv.
